# Прогнозирование стоимости автомобиля по характеристикам


Образовательная платформа: SkillFactory

Специализация: Data Science

Группа: DST-37 и 38

Юнит 6. Проект 5: "Выбираем автомобиль правильно"


### Задача:

    Создать модель, которая будет предсказывать стоимость автомобиля по его характеристикам для того, чтобы выявлять выгодные предложения (когда желаемая цена продавца ниже предсказанной рыночной цены).

### Метрика:

    MAPE (Mean Percentage Absolute Error) - средняя абсолютная ошибка в процентах

### Нужно:

    Составить train датасет - спарсить данные, либо найти готовый
    Обучить модель

### Плюс:

    Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
    Сгенерировать новые признаки
    Подгрузить еще больше данных
    Попробовать подобрать параметры модели
    Попробовать разные алгоритмы и библиотеки ML
    Сделать Ансамбль моделей, Blending, Stacking

### Этапы работы:

    Парсинг с авто.ру - Вадим, Евгений, Артём
    EDA, Feature Engineering - Артём, Вадим, Евгений
    Сравнение одиночных моделей - Артём, Вадим
    Стекинг - Вадим
    
    
#### В данном ноутбуке мы выполняем подготовку данных.

#### Также в этом проекте мы использовали:

Ноутбук, через который пытались парсить: https://www.kaggle.com/artemskakun/sf-dst-car-price-prediction-autoruparser

Спарсенный датасет мы взяли у этой команды, потому что парсинг занимал очень много времени: https://www.kaggle.com/juliadeinego/data-car-sales

Ноутбук, в котором провели ML: https://www.kaggle.com/artemskakun/sf-dst-car-price-prediction-ml

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import json
import csv
from datetime import datetime
from ast import literal_eval

import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.max_columns', None)

RANDOM_SEED = 42

AUTORU_DATA = '../input/data-parsing'
TEST_DATA = '../input/sf-dst-car-price-prediction/'

In [ ]:
# Определение функций
def normalise_doors(row):
    num = row['number_of_doors']
    if pd.isna(num) or num == 0:
        result = 4
        vehicle = row['body_type']
        if vehicle == 'минивэн':
            result = 5
        elif vehicle == 'седан':
            result = 4
        elif vehicle == 'купе':
            result = 2
        return result
    return num

def normalise_model_name(row):
    name = ''
    try:
        d = literal_eval(row['equipment_dict'])
        name = d['model']
    except Exception as e: 
        try:
            val = row['model_name'].split('|')
            if len(val)>1:
                name = val[0]
        except Exception as e2:
            name = row['model_name']
    return name


def normalise_brand_name(row):
    name = ''
    try:
        d = literal_eval(row['equipment_dict'])
        name = d['mark']
    except Exception as e: 
        name = row['brand']
    return name

def normalise_model_date(row):
    val = ''
    try:
        d = literal_eval(row['equipment_dict'])
        val = d['year']
    except Exception as e:
        val = row['production_date']
    finally: #but this work
        try:
            i = int(val)
            return i
        except Exception as e2:
            return 0
        
def normalise_electric_engine(df, row):
    power = row['engine_power']
    mean = df[(df['fuel_type']!='электро') \
                 & (df['engine_power']>=power*0.8) \
                 & (df['engine_power']<=power*1.2)].engine_displacement.mean()
    return int(round(mean, -3))
    
    
rates = {
    '20210416' : 76.9808,
    '20210417' : 75.5535,
    '20210511' : 74.1373,
    '20201021' : 77.7780,
    '20210415' : 75.6826,
    '20201020' : 77.9241,
    '20201019' : 77.9644,
    '20201025' : 76.4667,
    '20201024' : 76.4667,
    '20201026' : 76.4667
}

def price_exchange(row, column, exchange_to='d'):
    rate = rates.get(row['parcing_date'])
    if exchange_to=='d':
        return int(row[column] / rate)
    return int(row[column] * rate)

def remove_whitespace(x):
    try:
        x = "".join(x.split())
    except:
        pass
    return x

def count_words(string, good_words, bad_words):
    i = 0
    try:
        for word in string:
            if type(word) == float:
                continue
            try:
                if word in good_words:
                    i += 1
                elif word in bad_words:
                    i -= 1
            except:
                print(word)
    except:
        pass
    return(i)

### Считываем все файлы и объединяем в единый датафрейм

In [ ]:
print('List of data files in directory "data" ->',glob.glob('../input/data-parsing'))

required_columns = ['sell_id','parsing_unixtime','price','bodyType','color','engineDisplacement','enginePower',
                    'fuelType','mileage','productionDate','vehicleTransmission','Владельцы','ПТС','Руль',
                    'Состояние','brand','model_name','equipment_dict','description']

full_df = pd.DataFrame()
for file in glob.glob('../input/data-parsing/*.csv'):
    file_name = file[file.rindex('/')+1:]
    print(file_name)
    df = pd.DataFrame()
    try:
        df = pd.read_csv(file)
    except:
        df = pd.read_csv(file, delimiter=';')
    df = df[required_columns]
    full_df = pd.concat([full_df, df], ignore_index=True)
    
full_df.sample(2)


### Удалим дубликаты и пустые строки из спарсенных данных

In [ ]:
full_df = full_df.drop_duplicates(subset=['sell_id'])
full_df = full_df.dropna(how='all')
len(full_df)

### Оставляем только необходимые признаки

In [ ]:
# создаем список с новым названием колонок
new_columns = ['sell_id','parsing_unixtime','price','body_type','color','engine_displacement','engine_power',
               'fuel_type','mileage','production_date','vehicle_transmission','owners','vehicle_pasport',
               'wheel','condition','brand','model_name','equipment_dict','description']
        
# оставляем в датасете выбранные колонки
full_df = full_df[required_columns]

# переименуем колонки
full_df.columns = new_columns

full_df.sample(2)

### Проведём первичную обработку датасета

In [ ]:
def preprocessing_autoru_df(df):
    df.drop(['sell_id'], axis=1, inplace=True)
    # model date
    df['production_date'] = df['production_date'].fillna(0)
    df['model_date'] = df.apply(lambda row : normalise_model_date(row), axis=1)
    df['model_name'] = df.apply(lambda row : normalise_model_name(row), axis=1)
    df['brand'] = df.apply(lambda row : normalise_brand_name(row), axis=1)
    df.drop(df[(df['model_name'].isna()) & (full_df['brand'].isna())].index, inplace=True)
    
    df.drop('equipment_dict', axis=1, inplace=True)

    df['mileage'] = df['mileage'].fillna('0')
    df['mileage'] = df['mileage'].apply(remove_whitespace)
    df['mileage'] = df['mileage'].str.extract('(\d+)', expand=True)
    df['mileage'] = df['mileage'].fillna(0).astype(int)

    
preprocessing_autoru_df(full_df)
full_df.head(2)

### Загружаем тестовый датасет

In [ ]:
test = pd.read_csv('../input/sf-dst-car-price-prediction/test.csv')
test.sample(2)

In [ ]:
# выберем интересующие нас признаки
columns = ['bodyType', 'brand', 'color', 'engineDisplacement', 'enginePower',
           'fuelType', 'mileage', 'modelDate', 'model_name', 'numberOfDoors', 'parsing_unixtime',
           'productionDate', 'vehicleTransmission', 'Владельцы', 'ПТС', 'Руль', 'Состояние', 'description']

df_test = test[columns]

# создаем список с новым названием признаков
new_columns = ['body_type', 'brand', 'color', 'engine_displacement', 'engine_power',
               'fuel_type', 'mileage', 'model_date', 'model_name', 'number_of_doors', 'parsing_unixtime',
               'production_date', 'vehicle_transmission', 'owners', 'vehicle_pasport',
               'wheel', 'condition', 'description']

df_test.columns = new_columns

df_test.sample(2)

### Объединим оба датасета и проведем дальнейшую обработку одновременно

In [ ]:
full_df['test'] = 0
df_test['test'] = 1
df_test['price'] = 0
full_df = pd.concat([full_df, df_test], ignore_index=True)

### Проведем дальнейшую предобработку для объединенного датафрейма

In [ ]:
# найдём и удалим строку с неверными значениями
a = full_df[full_df['parsing_unixtime'] == 'https://avatars.mds.yandex.net/get-autoru-vos/2090686/56351fff854ba117076587162cfc6805/1200x900n'].index
full_df = full_df.drop(a, axis=0)

def preprocessing_full_df(df):
    # убираем пустые строки
    df.dropna(axis=1, how='all', inplace=True)
    
    # обрабатываем признак engine_power 
    df['engine_power'] = df['engine_power'].astype(str).str.extract('(\d+)', expand=True)\
        .fillna(0).astype(int)
    
    # обрабатываем признак engine_displacement
    df['engine_displacement'] = ((df['engine_displacement']).astype(str)\
        .str.extract('([0-9.]+)', expand=True).astype(float)\
        .fillna(0)*1000).astype(int)
    df['engine_displacement'] = df.apply(lambda row : normalise_electric_engine(df, row) \
        if (row.fuel_type=='электро') | (row.engine_displacement==0) else row.engine_displacement , axis=1)
    
    # обрабатываем признак number_of_doors
    df['number_of_doors'] = df.apply(lambda row : normalise_doors(row), axis=1).astype(int).apply(str)

    # обрабатываем признак body_type
    df['body_type'] = df['body_type'].fillna('седан')\
        .astype(str).apply(lambda x: x[:x.find(" ")] if x.find(" ") > 0 else x)
    
    # обрабатываем признак fuel_type
    df['fuel_type'] = df['fuel_type'].fillna('бензин').str.strip()\
        .replace({"бензин, газобаллонное оборудование": "бго", 
                  "газ, газобаллонное оборудование": "газ",
                  "дизель, газобаллонное оборудование": "дизель",
                  "гибрид, газобаллонное оборудование": "гибрид"}, inplace=True)
    
    # обрабатываем признак owners
    df['owners'] = df['owners'].astype(str)\
        .str.extract('(\d+)', expand=True)\
        .fillna('1').astype(int)

    # обрабатываем признак price
    df['price'] = df['price'].apply(remove_whitespace)\
        .str.extract('(\d+)', expand=True)\
        .fillna(0).astype(int)

    # обрабатываем признак vehicle_pasport
    df['vehicle_pasport'] = df['vehicle_pasport'].fillna('Оригинал')

    # обрабатываем признак production_date
    df['production_date'] = df['production_date'].fillna(df['production_date'].median()).astype(int)
    df['model_date'] = df['model_date'].fillna(value=df['production_date']).astype(int)
    
    # переводим признаки в нижний регистр
    for col in ['body_type','color','fuel_type','vehicle_transmission','vehicle_pasport',
                'wheel','brand']:
        df[col] = df[col].astype(str).str.lower()
    
    # переводим model_name в верхний регистр
    df['model_name'] = df['model_name'].astype(str).str.upper()

    # удаляем лишние признаки
    df.drop(df[df['condition'] == 'Битый / не на ходу'].index, inplace=True)
    df.drop('parsing_unixtime', axis=1, inplace=True)
    df.drop('production_date', axis=1, inplace=True)

preprocessing_full_df(full_df)
full_df.sample(2)

In [ ]:
full_df.info()

### Обработаем текстовое поле description

Попытаемся получить дополнительный признак о наличии положительных и отрицательных слов в тексте объявления

In [ ]:
words_dct = {}
for description in full_df['description']:
    if type(description) == float:
        continue
    description = description.lower()
    description = description.replace('!', '').replace('.', '').replace(',', '')
    description = description.split(' ')
    for word in description:
        if word in words_dct:
            words_dct[word] += 1
        else:
            words_dct[word] = 1
words_dct

In [ ]:
# сделаем два списка: положительные и отрицательные слова
good_words = ['комплект', 'официального', 'салон', 'птс', 'отличном', 'выгода', 'гарантия',
             'комплексную', 'оригинал', 'гарантию', 'официальный', 'подарок', 'гарантии',
              'новая', 'сертифицированными', '✅', 'предпродажную', 'новые', 'favorit',
              'отличное', 'надежный', 'официальным', 'отличный', 'ремонтировалась', 'хорошем']
bad_words = ['дтп', 'срочный', 'трейд-ин', 'ремонтов', 'ремонта', 'капремонта',
             'ремонтных', 'дтпвсе', 'авария', 'аварии']

# заполним поле description цифровыми данными
full_df['description'] = full_df['description'].apply(lambda x: count_words(x, good_words, bad_words))

### Сохраним подготовленные для МЛ данные в файл preproc.csv

In [ ]:
full_df.to_csv('preproc.csv', index=False)